In [ ]:
import sys
sys.path.insert(0, '../../scripts')

In [ ]:
from build_dependency_tree import *
from create_dot_files import *

In [ ]:
trimmed_graph()
full_graph()

In [ ]:
tree = build_tree()

In [ ]:
internal_packages = tuple(tree.keys())
internal_packages

In [ ]:
dag = nx.DiGraph()

for key, values in tree.items():
    dag.add_node(key)
    dag.add_nodes_from(values['internal'])
    dag.add_nodes_from(values['external'])
    edge_tuples = [
        (key, value) for value in values['internal']
    ]
    dag.add_edges_from(edge_tuples)
    edge_tuples = [
        (key, value) for value in values['external']
    ]
    dag.add_edges_from(edge_tuples)
    
topological = list(nx.topological_sort(dag))
topological

In [ ]:
level_map = {}
for package in topological[::-1]:
    if package not in internal_packages:
        level_map[package] = 0
    else:
        depencencies = nx.descendants(dag, package)
        levels = {0}
        for dependency in depencencies:
            if dependency in internal_packages:
                try:
                    levels.add(level_map[dependency])
                except KeyError:
                    pass
        max_level = max(levels)
        level_map[package] = max_level + 1

In [ ]:
levels = {
    level: []
    for level in range(max(level_map.values()) + 1)
}
for package, level in level_map.items():
    levels[level].append(package)
    
levels

In [ ]:
clusters = ""

for level in range(max(level_map.values()) + 1):
    cluster_packages = ';\n        '.join(levels[level])
    clusters += """
    subgraph cluster_{0} {{
        {1};
        label = "Level {0}";
        style = dashed;
        color = grey80;
    }}
    """.format(level, cluster_packages)

edges = ""

for edge in dag.edges():
    edges += "    {} -> {};\n".format(*edge)

dot_file_contents = """
strict digraph  {{

    rankdir = LR;
    node [
        shape = box;
        width = 3;
    ];
    splines = spline;

{}
{}
}}
""".format(clusters, edges)

print(dot_file_contents)